In [474]:
import pandas as pd
import numpy as np

In [778]:
from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import IUPAC
import sys
# import pyensembl
import os
#https://towardsdatascience.com/a-simple-guide-to-command-line-arguments-with-argparse-6824c30ab1c3
import warnings
warnings.filterwarnings("ignore")

In [476]:
# SK: Dictionary of protein names to sequences
proteins = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_translations.fa", "fasta"):
    name = record.id.split("|")[1].split(".")[0]
    proteins[name] = str(record.seq)
    

# SK: Dictionary of protein names to CDS dna transcript

dna_transcripts = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_transcripts.fa", "fasta"):
	name = record.id.split("|")[0].split(".")[0]
	record_c = record.id.split("|")
	for i in record_c:
		if "CDS" in i:
			coords = i.replace("CDS:","")
	start = int(coords.split("-")[0])
	end = int(coords.split("-")[1])
	dna_seq = str(record.seq)[start-1:end]
	dna_transcripts[name] = dna_seq

# Plan:
1. Use cds coordinates, create df with genomic coordinates as one column and corresponding nt as second column
2. Use position in variant file to resplace the value in the nt column
3. Translate the nt column

In [748]:
uniprotID = "P11473"



In [749]:


uniprotID_ENST_mapping = pd.read_csv("../../data/SFARI_TFs_with_ENST.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["uniprotID"],uniprotID_ENST_mapping["ENST"]))
uniprotID_ENST_mapping_dict['O60479'] = 'ENST00000434704'
ENST = uniprotID_ENST_mapping_dict[uniprotID]



In [750]:
variants = pd.read_csv("../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/" + ENST + ".bed", sep = "\t", header = None)
no_syn_variants = variants[variants[22] == "No-Syn"]
no_syn_variants

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,12,47844749,47844750,AD,ENSG00000111424,.,-,ENST00000395324,12,47844749,...,-,12,47844749,47844750,G,A,0.000005,S,F,No-Syn
1,12,47844758,47844759,AD,ENSG00000111424,.,-,ENST00000395324,12,47844758,...,-,12,47844758,47844759,T,C,0.000005,N,S,No-Syn
12,12,47855696,47855697,AD,ENSG00000111424,.,-,ENST00000395324,12,47855696,...,-,12,47855696,47855697,G,C,0.000028,L,V,No-Syn
14,12,47855719,47855720,AD,ENSG00000111424,.,-,ENST00000395324,12,47855719,...,-,12,47855719,47855720,G,C,0.000005,S,C,No-Syn
15,12,47855720,47855721,AD,ENSG00000111424,.,-,ENST00000395324,12,47855720,...,-,12,47855720,47855721,A,C,0.000005,S,A,No-Syn
17,12,47855724,47855725,AD,ENSG00000111424,.,-,ENST00000395324,12,47855724,...,-,12,47855724,47855725,C,G,0.000014,E,D,No-Syn
19,12,47855741,47855742,AD,ENSG00000111424,.,-,ENST00000395324,12,47855741,...,-,12,47855741,47855742,G,A,0.000009,P,S,No-Syn
20,12,47855742,47855743,AD,ENSG00000111424,.,-,ENST00000395324,12,47855742,...,-,12,47855742,47855743,G,T,0.000014,D,E,No-Syn
21,12,47855743,47855744,AD,ENSG00000111424,.,-,ENST00000395324,12,47855743,...,-,12,47855743,47855744,T,C,0.000028,D,G,No-Syn
26,12,47855752,47855753,AD,ENSG00000111424,.,-,ENST00000395324,12,47855752,...,-,12,47855752,47855753,T,C,0.000014,D,G,No-Syn


In [751]:
strand = no_syn_variants[13].iloc[1]
strand

'-'

In [752]:
wt_nt_seq = dna_transcripts[ENST]
wt_AA_seq = str(Seq(wt_nt_seq).translate())
wt_nt_seq

'ATGGAGGCAATGGCGGCCAGCACTTCCCTGCCTGACCCTGGAGACTTTGACCGGAACGTGCCCCGGATCTGTGGGGTGTGTGGAGACCGAGCCACTGGCTTTCACTTCAATGCTATGACCTGTGAAGGCTGCAAAGGCTTCTTCAGGCGAAGCATGAAGCGGAAGGCACTATTCACCTGCCCCTTCAACGGGGACTGCCGCATCACCAAGGACAACCGACGCCACTGCCAGGCCTGCCGGCTCAAACGCTGTGTGGACATCGGCATGATGAAGGAGTTCATTCTGACAGATGAGGAAGTGCAGAGGAAGCGGGAGATGATCCTGAAGCGGAAGGAGGAGGAGGCCTTGAAGGACAGTCTGCGGCCCAAGCTGTCTGAGGAGCAGCAGCGCATCATTGCCATACTGCTGGACGCCCACCATAAGACCTACGACCCCACCTACTCCGACTTCTGCCAGTTCCGGCCTCCAGTTCGTGTGAATGATGGTGGAGGGAGCCATCCTTCCAGGCCCAACTCCAGACACACTCCCAGCTTCTCTGGGGACTCCTCCTCCTCCTGCTCAGATCACTGTATCACCTCTTCAGACATGATGGACTCGTCCAGCTTCTCCAATCTGGATCTGAGTGAAGAAGATTCAGATGACCCTTCTGTGACCCTAGAGCTGTCCCAGCTCTCCATGCTGCCCCACCTGGCTGACCTGGTCAGTTACAGCATCCAAAAGGTCATTGGCTTTGCTAAGATGATACCAGGATTCAGAGACCTCACCTCTGAGGACCAGATCGTACTGCTGAAGTCAAGTGCCATTGAGGTCATCATGTTGCGCTCCAATGAGTCCTTCACCATGGACGACATGTCCTGGACCTGTGGCAACCAAGACTACAAGTACCGCGTCAGTGACGTGACCAAAGCCGGACACAGCCTGGAGCTGATTGAGCCCCTCATCAAGTTCCAGGTGGGACTGAAGAAGCTGAACTTGCATGAGGAGGAGCATGTCCTGCTC

In [753]:
if strand == "-":
    wt_nt_seq = str(Seq(wt_nt_seq).complement())
wt_nt_seq

'TACCTCCGTTACCGCCGGTCGTGAAGGGACGGACTGGGACCTCTGAAACTGGCCTTGCACGGGGCCTAGACACCCCACACACCTCTGGCTCGGTGACCGAAAGTGAAGTTACGATACTGGACACTTCCGACGTTTCCGAAGAAGTCCGCTTCGTACTTCGCCTTCCGTGATAAGTGGACGGGGAAGTTGCCCCTGACGGCGTAGTGGTTCCTGTTGGCTGCGGTGACGGTCCGGACGGCCGAGTTTGCGACACACCTGTAGCCGTACTACTTCCTCAAGTAAGACTGTCTACTCCTTCACGTCTCCTTCGCCCTCTACTAGGACTTCGCCTTCCTCCTCCTCCGGAACTTCCTGTCAGACGCCGGGTTCGACAGACTCCTCGTCGTCGCGTAGTAACGGTATGACGACCTGCGGGTGGTATTCTGGATGCTGGGGTGGATGAGGCTGAAGACGGTCAAGGCCGGAGGTCAAGCACACTTACTACCACCTCCCTCGGTAGGAAGGTCCGGGTTGAGGTCTGTGTGAGGGTCGAAGAGACCCCTGAGGAGGAGGAGGACGAGTCTAGTGACATAGTGGAGAAGTCTGTACTACCTGAGCAGGTCGAAGAGGTTAGACCTAGACTCACTTCTTCTAAGTCTACTGGGAAGACACTGGGATCTCGACAGGGTCGAGAGGTACGACGGGGTGGACCGACTGGACCAGTCAATGTCGTAGGTTTTCCAGTAACCGAAACGATTCTACTATGGTCCTAAGTCTCTGGAGTGGAGACTCCTGGTCTAGCATGACGACTTCAGTTCACGGTAACTCCAGTAGTACAACGCGAGGTTACTCAGGAAGTGGTACCTGCTGTACAGGACCTGGACACCGTTGGTTCTGATGTTCATGGCGCAGTCACTGCACTGGTTTCGGCCTGTGTCGGACCTCGACTAACTCGGGGAGTAGTTCAAGGTCCACCCTGACTTCTTCGACTTGAACGTACTCCTCCTCGTACAGGACGAG

In [754]:
nt_df = pd.DataFrame({"nt" : [*wt_nt_seq]})
nt_df

,nt
0,T
1,A
2,C
3,C
4,T
...,...
1279,G
1280,G
1281,A
1282,C


In [755]:
cds_bed = pd.read_csv("../outputs/mutations/cds_bed_format/" + ENST, sep = "\t", header = None)
cds_bed = cds_bed[[1, 2]]
cds_bed[1] += 1
cds_bed

,1,2
0,47878968,47879113
1,47865047,47865177
2,47857504,47857688
3,47857129,47857249
4,47855630,47855801
5,47846657,47846808
6,47846335,47846451
7,47844746,47845005


In [756]:
if strand == "-":
    cds_bed = cds_bed.sort_values(by = 1, ascending = False)
else:
    cds_bed = cds_bed.sort_values(by = 1, ascending = True)
cds_bed

,1,2
0,47878968,47879113
1,47865047,47865177
2,47857504,47857688
3,47857129,47857249
4,47855630,47855801
5,47846657,47846808
6,47846335,47846451
7,47844746,47845005


In [757]:
range_col = []
for start, end in zip(cds_bed[1], cds_bed[2]):
    if strand == "-":
        range_col += list(range(end, start - 1, -1))
    else:
        range_col += list(range(start, end + 1))

In [758]:
len(range_col)

1284

In [759]:
nt_df["gen_pos"] = range_col
nt_df = nt_df.set_index("gen_pos")
nt_df

,nt
gen_pos,
47879113,T
47879112,A
47879111,C
47879110,C
47879109,T
...,...
47844750,G
47844749,G
47844748,A


In [760]:
# nt_df.loc[37020709]

In [761]:
# nt_df.loc[37020742]

In [762]:
# nt_df.loc[37015013]

In [763]:
# nt_df.loc[37015049]

In [764]:
no_syn_variants

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,12,47844749,47844750,AD,ENSG00000111424,.,-,ENST00000395324,12,47844749,...,-,12,47844749,47844750,G,A,0.000005,S,F,No-Syn
1,12,47844758,47844759,AD,ENSG00000111424,.,-,ENST00000395324,12,47844758,...,-,12,47844758,47844759,T,C,0.000005,N,S,No-Syn
12,12,47855696,47855697,AD,ENSG00000111424,.,-,ENST00000395324,12,47855696,...,-,12,47855696,47855697,G,C,0.000028,L,V,No-Syn
14,12,47855719,47855720,AD,ENSG00000111424,.,-,ENST00000395324,12,47855719,...,-,12,47855719,47855720,G,C,0.000005,S,C,No-Syn
15,12,47855720,47855721,AD,ENSG00000111424,.,-,ENST00000395324,12,47855720,...,-,12,47855720,47855721,A,C,0.000005,S,A,No-Syn
17,12,47855724,47855725,AD,ENSG00000111424,.,-,ENST00000395324,12,47855724,...,-,12,47855724,47855725,C,G,0.000014,E,D,No-Syn
19,12,47855741,47855742,AD,ENSG00000111424,.,-,ENST00000395324,12,47855741,...,-,12,47855741,47855742,G,A,0.000009,P,S,No-Syn
20,12,47855742,47855743,AD,ENSG00000111424,.,-,ENST00000395324,12,47855742,...,-,12,47855742,47855743,G,T,0.000014,D,E,No-Syn
21,12,47855743,47855744,AD,ENSG00000111424,.,-,ENST00000395324,12,47855743,...,-,12,47855743,47855744,T,C,0.000028,D,G,No-Syn
26,12,47855752,47855753,AD,ENSG00000111424,.,-,ENST00000395324,12,47855752,...,-,12,47855752,47855753,T,C,0.000014,D,G,No-Syn


In [765]:
# Checking 
for i in no_syn_variants.index:
    pos = no_syn_variants[2].loc[i]
    sfari_nt = no_syn_variants[no_syn_variants[2] == pos][17].loc[i]
    nt_df_nt = nt_df.loc[pos].iloc[0]
    if sfari_nt != nt_df_nt:
        print("mismatch!")

In [766]:
no_syn_variants[np.arange(12)]

,0,1,2,3,4,5,6,7,8,9,10,11
0,12,47844749,47844750,AD,ENSG00000111424,.,-,ENST00000395324,12,47844749,47844750,ENSG00000111424
1,12,47844758,47844759,AD,ENSG00000111424,.,-,ENST00000395324,12,47844758,47844759,ENSG00000111424
12,12,47855696,47855697,AD,ENSG00000111424,.,-,ENST00000395324,12,47855696,47855697,ENSG00000111424
14,12,47855719,47855720,AD,ENSG00000111424,.,-,ENST00000395324,12,47855719,47855720,ENSG00000111424
15,12,47855720,47855721,AD,ENSG00000111424,.,-,ENST00000395324,12,47855720,47855721,ENSG00000111424
17,12,47855724,47855725,AD,ENSG00000111424,.,-,ENST00000395324,12,47855724,47855725,ENSG00000111424
19,12,47855741,47855742,AD,ENSG00000111424,.,-,ENST00000395324,12,47855741,47855742,ENSG00000111424
20,12,47855742,47855743,AD,ENSG00000111424,.,-,ENST00000395324,12,47855742,47855743,ENSG00000111424
21,12,47855743,47855744,AD,ENSG00000111424,.,-,ENST00000395324,12,47855743,47855744,ENSG00000111424
26,12,47855752,47855753,AD,ENSG00000111424,.,-,ENST00000395324,12,47855752,47855753,ENSG00000111424


In [767]:
no_syn_variants[np.arange(12, 23)]

,12,13,14,15,16,17,18,19,20,21,22
0,ENST00000395324,-,12,47844749,47844750,G,A,0.000005,S,F,No-Syn
1,ENST00000395324,-,12,47844758,47844759,T,C,0.000005,N,S,No-Syn
12,ENST00000395324,-,12,47855696,47855697,G,C,0.000028,L,V,No-Syn
14,ENST00000395324,-,12,47855719,47855720,G,C,0.000005,S,C,No-Syn
15,ENST00000395324,-,12,47855720,47855721,A,C,0.000005,S,A,No-Syn
17,ENST00000395324,-,12,47855724,47855725,C,G,0.000014,E,D,No-Syn
19,ENST00000395324,-,12,47855741,47855742,G,A,0.000009,P,S,No-Syn
20,ENST00000395324,-,12,47855742,47855743,G,T,0.000014,D,E,No-Syn
21,ENST00000395324,-,12,47855743,47855744,T,C,0.000028,D,G,No-Syn
26,ENST00000395324,-,12,47855752,47855753,T,C,0.000014,D,G,No-Syn


In [768]:
no_syn_variants["name"] = "g." + no_syn_variants[2].astype(str) + no_syn_variants[17] + ">" + no_syn_variants[18]
no_syn_variants

<ipython-input-768-a4cc02e035aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_syn_variants["name"] = "g." + no_syn_variants[2].astype(str) + no_syn_variants[17] + ">" + no_syn_variants[18]


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,name
0,12,47844749,47844750,AD,ENSG00000111424,.,-,ENST00000395324,12,47844749,...,12,47844749,47844750,G,A,0.000005,S,F,No-Syn,g.47844750G>A
1,12,47844758,47844759,AD,ENSG00000111424,.,-,ENST00000395324,12,47844758,...,12,47844758,47844759,T,C,0.000005,N,S,No-Syn,g.47844759T>C
12,12,47855696,47855697,AD,ENSG00000111424,.,-,ENST00000395324,12,47855696,...,12,47855696,47855697,G,C,0.000028,L,V,No-Syn,g.47855697G>C
14,12,47855719,47855720,AD,ENSG00000111424,.,-,ENST00000395324,12,47855719,...,12,47855719,47855720,G,C,0.000005,S,C,No-Syn,g.47855720G>C
15,12,47855720,47855721,AD,ENSG00000111424,.,-,ENST00000395324,12,47855720,...,12,47855720,47855721,A,C,0.000005,S,A,No-Syn,g.47855721A>C
17,12,47855724,47855725,AD,ENSG00000111424,.,-,ENST00000395324,12,47855724,...,12,47855724,47855725,C,G,0.000014,E,D,No-Syn,g.47855725C>G
19,12,47855741,47855742,AD,ENSG00000111424,.,-,ENST00000395324,12,47855741,...,12,47855741,47855742,G,A,0.000009,P,S,No-Syn,g.47855742G>A
20,12,47855742,47855743,AD,ENSG00000111424,.,-,ENST00000395324,12,47855742,...,12,47855742,47855743,G,T,0.000014,D,E,No-Syn,g.47855743G>T
21,12,47855743,47855744,AD,ENSG00000111424,.,-,ENST00000395324,12,47855743,...,12,47855743,47855744,T,C,0.000028,D,G,No-Syn,g.47855744T>C
26,12,47855752,47855753,AD,ENSG00000111424,.,-,ENST00000395324,12,47855752,...,12,47855752,47855753,T,C,0.000014,D,G,No-Syn,g.47855753T>C


In [769]:
names = []
TF_seqs = []
prot_positions = []

In [770]:
for i in no_syn_variants.index:
    var_pos = no_syn_variants[2].loc[i]
    wt_nt = no_syn_variants[17].loc[i]
    var_nt = no_syn_variants[18].loc[i]
    g_name = no_syn_variants["name"].loc[i]

    nt_df_var_copy = nt_df.copy(deep = True)

    if nt_df_var_copy.at[var_pos, "nt"] == wt_nt:
        nt_df_var_copy.at[var_pos, "nt"] = var_nt
    else:
        print("mismatch!")

    new_nt_seq = "".join(nt_df_var_copy["nt"])
    if strand == "-":
        new_nt_seq = Seq(new_nt_seq).complement()
    new_AA_seq = str(Seq(new_nt_seq).translate())

    prot_pos = 0
    for i in range(len(new_AA_seq)):
        if new_AA_seq[i] != wt_AA_seq[i]:
            prot_change_descrip = "p." + str(i + 1)+ new_AA_seq[i] + "REP" + wt_AA_seq[i]
            prot_pos = i + 1

    prot_positions.append(prot_pos)
    names.append(g_name + "(" + prot_change_descrip + ")")
    TF_seqs.append(new_AA_seq)

In [771]:
variant_TF_seqs_df = pd.DataFrame({"name": names,
             "TF_seq": TF_seqs, 
            "prot_pos":prot_positions})
variant_TF_seqs_df

,name,TF_seq,prot_pos
0,g.47844750G>A(p.427FREPS),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,427
1,g.47844759T>C(p.424SREPN),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,424
2,g.47855697G>C(p.230VREPL),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,230
3,g.47855720G>C(p.222CREPS),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,222
4,g.47855721A>C(p.222AREPS),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,222
5,g.47855725C>G(p.220DREPE),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,220
6,g.47855742G>A(p.215SREPP),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,215
7,g.47855743G>T(p.214EREPD),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,214
8,g.47855744T>C(p.214GREPD),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,214
9,g.47855753T>C(p.211GREPD),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,211


In [772]:
def return_AD_seqs(AD_start, AD_end):
    within_AD = variant_TF_seqs_df[(AD_start <= variant_TF_seqs_df["prot_pos"]) & (variant_TF_seqs_df["prot_pos"] <= AD_end)]
    within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]
    return within_AD

In [773]:
return_AD_seqs(340, 477)

<ipython-input-772-ea883f5a61cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]


,name,TF_seq,prot_pos,AD_seq
0,g.47844750G>A(p.427FREPS),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,427,SPDRPGVQDAALIEAIQDRLSNTLQTYIRCRHPPPGSHLLYAKMIQ...
1,g.47844759T>C(p.424SREPN),MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,424,SPDRPGVQDAALIEAIQDRLSNTLQTYIRCRHPPPGSHLLYAKMIQ...


In [774]:
known_ADs = pd.read_csv("../../output/known_ADs_considering_isoforms_and_canonical.csv")
AD_rows = known_ADs[known_ADs["uniprotID"] == uniprotID]
AD_rows

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
691,VDR,415,427,P11473,[['P11473']],NaN,P11473,"PMID: 15908514, Soto",TF,TPLVLEVFGNEIS,NaN
692,VDR,195,238,P11473,"[['P11473'], ['P11473']]",nan / nan,P11473 / P11473,"Choi 2000 list OR uniprot. check, GSL / Stalle...",TF,DMMDSSSFSNLDLSEEDSDDPSVTLELSQLSMLPHLADLVSYSI,NaN


In [775]:
gene = AD_rows["Gene"].iloc[0]

In [776]:
def save_fasta(start, end):
    df = return_AD_seqs(start, end)
    ofile = open("../outputs/AD_variant_fasta/" + gene + "_" + uniprotID + "_" + "AD_" + str(start) + "-" + str(end), "w")
    for i in df.index:
        ofile.write(">" + df["name"].loc[i] + "\n" + df["AD_seq"].loc[i] + "\n")
    ofile.close()

In [777]:
for start, end in zip(AD_rows["Start"], AD_rows["End"]):
    save_fasta(start, end)

<ipython-input-772-ea883f5a61cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]
<ipython-input-772-ea883f5a61cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]
